## Connect and query

In [71]:
import sys
sys.path.append('/home/delgado/proj/buhayra_manuscript/buhayra')


In [72]:
from credentials import *

In [73]:
import psycopg2
import osgeo.ogr
import shapely
import shapely.wkt
import geopandas as gpd

In [152]:
connection = psycopg2.connect(database="watermasks",user=postgis_user, password=postgis_pass,host=postgis_host)
cursor = connection.cursor()



## Derive query
The syntax of the query should look something like:

```
SELECT `id`, `id_jrc`, `ingestion_time`, `area`
FROM (SELECT `id`, `id_jrc`, `ingestion_time`, `area`, MAX(`ingestion_time`) OVER (PARTITION BY `id_jrc`) AS `q01`
FROM `df`)
WHERE (`ingestion_time` = `q01`)
```

I am splitting the query into a sub query `sub_q` and a main query. ***In the end I used only the subquery and performed the final filter on geopandas***

In [91]:
year=2020
month=4
 
sub_q_select = " SELECT id, id_jrc, ingestion_time, area, ST_AsText(geom), MAX(ingestion_time) OVER (PARTITION BY id_jrc)"
sub_q_from = " FROM manuscript_threshold_geom"
sub_q_where = " WHERE area>0 AND to_char(ingestion_time,'YYYY')='" + "{:0>2d}".format(year) + "' AND to_char(ingestion_time,'MM')='"+ "{:0>2d}".format(month) +"'"


sub_q = sub_q_select + sub_q_from + sub_q_where

#q_select = " SELECT id, id_jrc, ingestion_time, area, ST_AsText(geom)"
#q_from = " FROM (" + sub_q + ") AS q01"
#q_where=" WHERE (ingestion_time = q01)"

#query=q_select + q_from + q_where + ' LIMIT 50;'



In [153]:
cursor.execute(sub_q)

In [94]:
cursor.description

(Column(name='id', type_code=20),
 Column(name='id_jrc', type_code=23),
 Column(name='ingestion_time', type_code=1184),
 Column(name='area', type_code=700),
 Column(name='st_astext', type_code=25),
 Column(name='max', type_code=1184))

In [154]:
rows_list=[]
for index,id_jrc,ingestion_time,area,geo,max_ingest in cursor:
    data={'id':index,'id_jrc':id_jrc,'ingestion_time':ingestion_time,'area':area,'geometry':shapely.wkt.loads(geo),'max':max_ingest}
    rows_list.append(data)
gdf=gpd.GeoDataFrame(rows_list,crs='epsg:4326').set_index(['id'])



### Filter only latest extents in the given month for each id_jrc

In [169]:
gdf_unique=gdf.loc[gdf['max']==gdf['ingestion_time']].copy()

In [173]:
gdf_unique

id_jrc            ingestion_time      area  \
id                                                   
182860       5 2020-04-04 08:26:04+02:00  11068.50   
172573       7 2020-04-11 08:17:35+02:00   8195.97   
161736      12 2020-04-11 08:17:35+02:00  44140.20   
165011      19 2020-04-11 08:17:35+02:00  40387.70   
165307      24 2020-04-11 08:17:35+02:00  13330.40   
...        ...                       ...       ...   
169957   28656 2020-04-11 08:17:35+02:00  13133.20   
144971   28667 2020-04-06 08:09:48+02:00   7406.89   
160529   28670 2020-04-11 08:17:35+02:00   4344.84   
159577   28680 2020-04-11 08:17:35+02:00  11849.50   
183001   28681 2020-04-04 08:26:04+02:00  14331.20   

                                                 geometry  \
id                                                          
182860  MULTIPOLYGON (((-40.68198 -5.88946, -40.68189 ...   
172573  MULTIPOLYGON (((-38.56555 -5.88977, -38.56528 ...   
161736  MULTIPOLYGON (((-38.53087 -5.88986, -38.53078 ...   
165011  MULTIPOLYGON (((-38.52306 -5.89157, -38.52261 ...   
165307  MULTIPOLYGON (((-38.65565 -5.89364, -38.65502 ...   
...                                                   ...   
169957  MULTIPOLYGON (((-38.64379 -5.88268, -38.64361 ...   
144971  MULTIPOLYGON (((-38.24954 -5.88628, -38.24936 ...   
160529  MULTIPOLYGON (((-38.59735 -5.88663, -38.59699 ...   
159577  MULTIPOLYGON (((-38.60624 -5.88780, -38.60606 ...   
183001  MULTIPOLYGON (((-40.77621 -5.88830, -40.77612 ...   

                             max  
id                                
182860 2020-04-04 08:26:04+02:00  
172573 2020-04-11 08:17:35+02:00  
161736 2020-04-11 08:17:35+02:00  
165011 2020-04-11 08:17:35+02:00  
165307 2020-04-11 08:17:35+02:00  
...                          ...  
169957 2020-04-11 08:17:35+02:00  
144971 2020-04-06 08:09:48+02:00  
160529 2020-04-11 08:17:35+02:00  
159577 2020-04-11 08:17:35+02:00  
183001 2020-04-04 08:26:04+02:00  

[12161 rows x 5 columns]

### Create list of geometries in shapely

In [209]:
gdf_unique["value"]=1
shapes = ((geom, value) for geom, value in zip(gdf_unique.geometry, gdf_unique.value))

### Create template raster

In [181]:
import rasterio
rst = rasterio.open('../data/benchmark.tif')
meta = rst.meta.copy()
meta.update(compress='lzw')


In [195]:
meta["transform"]

Affine(0.00025, 0.0, -40.0,
       0.0, -0.00025, -2.837)

### Burn into raster

In [208]:
from rasterio import features
with rasterio.open('../data/burned_raster.tif', 'w+', **meta) as out:
    out_arr = out.read(1)
#     burned = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=out.transform)
    burned = features.rasterize(shapes=shapes, out_shape=(meta['height'],meta['width']),fill=0, transform=meta['transform'])
    out.write_band(1, burned)

### Alternative without opening file

In [210]:
burned1 = features.rasterize(shapes=shapes, out_shape=[meta['height'],meta['width']],fill=0, transform=meta['transform'])
# burned2 = features.rasterize(shapes=shapes, out_shape=[meta['height'],meta['width']],fill=0, transform=meta['transform'])


### Still can save raster to file later

In [211]:
with rasterio.open('../data/burned_raster.tif', 'w+', **meta) as out:
    out.write_band(1,burned1)